<a href="https://colab.research.google.com/github/Dasika-Vaishnavi/Wave2Web_forecast/blob/main/Json_and_Updating_frequency.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings; 
warnings.simplefilter('ignore')

In [2]:
!pip install pandas
!pip install prophet

     |████████████████████████████████| 65 kB 2.1 MB/s 
     |████████████████████████████████| 49 kB 3.3 MB/s 
     |████████████████████████████████| 179 kB 9.3 MB/s 
  Created wheel for prophet: filename=prophet-1.0.1-py3-none-any.whl size=6640423 sha256=9a4ebe776a7c1b824789c899b9f9273c187c1273ad2dbce1a39a13c5e9918789
  Stored in directory: /root/.cache/pip/wheels/4e/a0/1a/02c9ec9e3e9de6bdbb3d769d11992a6926889d71567d6b9b67
Successfully built prophet
  Attempting uninstall: cmdstanpy
    Found existing installation: cmdstanpy 0.9.5
    Uninstalling cmdstanpy-0.9.5:
      Successfully uninstalled cmdstanpy-0.9.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fbprophet 0.7.1 requires cmdstanpy==0.9.5, but you have cmdstanpy 0.9.68 which is incompatible.


# 0. Install and import dependencies

In [3]:
import pandas as pd
from prophet import Prophet

# 1. Read and process data

In [5]:
df = pd.read_csv('/content/picchi_peaks.csv')
df['Year'] = df['FLOW_DATE'].apply(lambda x: str(x)[-4:])
df['Month'] = df['FLOW_DATE'].apply(lambda x: str(x)[-6:-4])
df['Day'] = df['FLOW_DATE'].apply(lambda x: str(x)[:-6])
df['ds'] = pd.DatetimeIndex(df['Day']+'-'+df['Year'])
df.drop(['FLOW_DATE', 'Year', 'Month', 'Day'], axis=1, inplace=True)
df.head()

,y,ds
0,46.42,2011-01-01
1,46.54,2011-01-01
2,46.64,2011-01-01
3,46.69,2011-01-01
4,46.70,2011-01-01


# 2. Train the model

In [6]:
m = Prophet(interval_width=0.95, daily_seasonality=True)
model = m.fit(df)

# 3. Forecasting the data

In [7]:
future = m.make_future_dataframe(periods=100,freq='D')
forecast = m.predict(future)
forecast.head()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,daily,daily_lower,daily_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2010-09-03,46.058854,45.279079,70.410992,46.058854,46.058854,11.620017,11.620017,11.620017,1.036155,1.036155,1.036155,0.193177,0.193177,0.193177,10.390685,10.390685,10.390685,0.0,0.0,0.0,57.678871
1,2010-12-03,42.335766,36.856962,61.460086,42.335766,42.335766,7.113038,7.113038,7.113038,1.036155,1.036155,1.036155,0.193177,0.193177,0.193177,5.883706,5.883706,5.883706,0.0,0.0,0.0,49.448804
2,2011-01-01,41.149287,33.815952,58.678003,41.149287,41.149287,4.769734,4.769734,4.769734,1.036155,1.036155,1.036155,0.076082,0.076082,0.076082,3.657498,3.657498,3.657498,0.0,0.0,0.0,45.919021
3,2011-01-02,41.108374,32.588800,57.822808,41.108374,41.108374,4.555793,4.555793,4.555793,1.036155,1.036155,1.036155,0.305051,0.305051,0.305051,3.214588,3.214588,3.214588,0.0,0.0,0.0,45.664167
4,2011-01-03,41.067461,33.897549,57.434181,41.067461,41.067461,4.011792,4.011792,4.011792,1.036155,1.036155,1.036155,0.351497,0.351497,0.351497,2.624141,2.624141,2.624141,0.0,0.0,0.0,45.079253


In [8]:
import json
from prophet.serialize import model_to_json, model_from_json

with open('serialized_model.json', 'w') as fout:
    json.dump(model_to_json(m), fout)  # Save model

with open('serialized_model.json', 'r') as fin:
    m = model_from_json(json.load(fin)) 

In [9]:
m = Prophet(growth='flat')


In [11]:
def stan_init(m):
    """Retrieve parameters from a trained model.
    
    Retrieve parameters from a trained model in the format
    used to initialize a new Stan model.
    
    Parameters
    ----------
    m: A trained model of the Prophet class.
    
    Returns
    -------
    A Dictionary containing retrieved parameters of m.
    
    """
    res = {}
    for pname in ['k', 'm', 'sigma_obs']:
        res[pname] = m.params[pname][0][0]
    for pname in ['delta', 'beta']:
        res[pname] = m.params[pname][0]
    return res
df1 = df.loc[df['ds'] < '2016-01-19', :]  # All data except the last day
m1 = Prophet().fit(df1) # A model fit to all data except the last day


%timeit m2 = Prophet().fit(df)  # Adding the last day, fitting from scratch
%timeit m2 = Prophet().fit(df, init=stan_init(m1))  # Adding the last day, warm-starting from m1


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


1 loop, best of 5: 1.83 s per loop


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


1 loop, best of 5: 1.73 s per loop
